<a href="https://colab.research.google.com/github/mr7495/Pistachio-Counting/blob/master/Pistachio_RetinaNet_Training_%26_Evaluating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Code from https://github.com/mr7495/Pistachio-Counting

In [0]:
# Downgrade the TensorFlow to 1.15 because the current version of RetinaNet does not support higher versions.
!pip uninstall tensorflow
!pip install tensorflow==1.15
!pip install keras --upgrade


In [0]:
!pip install git+https://github.com/mr7495/RetinaNet --upgrade

In [0]:
!git clone https://github.com/mr7495/RetinaNet

In [0]:
!git clone https://github.com/mr7495/Pistachio-Counting

In [0]:
cd RetinaNet

In [0]:
!python setup.py build_ext --inplace

In [0]:
cd ..

In [0]:
# Start Training
# The number 1 in train1 & test1 refers to the first fold. If you want to train on other folds like fold2 change train1 & test1 with train2 & test2
import keras.backend as k
k.clear_session()
!RetinaNet/keras_retinanet/bin/train.py --epoch=50 --step=1017 --batch-size=1  --random-transform --backbone='resnet152' csv 'Pistachio-Counting/pesteh_set/train1.csv' 'Pistachio-Counting/pesteh_set/classes.csv' --val-annotations='Pistachio-Counting/pesteh_set/test1.csv'

In [0]:
import keras
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
import cv2
import os
import numpy as np
import time
import tensorflow as tf
import csv
import shutil
import keras.backend as k
from keras_retinanet.preprocessing.csv_generator import CSVGenerator
from keras_retinanet.utils.eval import _get_detections,_get_annotations,evaluate

In [0]:
#Add the trained networks to the list
nets=[]
for r,d,f in os.walk('snapshots'):
  for file in f:
    if '.h5' in file:
      if name in file:
        nets.append(os.path.join(r,file))

In [0]:
#In the next Cell we evaluate the trained networks

In [0]:
#Evaluation Metrics :Recall, Precision, F1 score, accuracy for each class and all the classes.
#Also Average Precision(AP) for each of the classes and mean average precision(map) for all of th classes
data={'Net':['FOLD','AP_0','AP_1','f1_0','f1_1','acc_0','acc_1','recall_0','recall_1','prec_0','prec_1','tp_0',
      'tp_1','fp_0','fp_1','fn_0','fn_1','TP','FP','FN','AP','F1','ACC','RECALL','PREC','number_0','number_1','NUMBER']}

fold_num=2 #choose fold number
for model_name in nets:
  k.clear_session()
  if 'resnet50' in  model_name:
    bn='resnet50'
  elif 'resnet152' in  model_name:
    bn='resnet152'
  if 'vgg16' in  model_name:
    bn='vgg16'
  tmodel = models.load_model(model_name,backbone_name=bn)
  model = models.convert_model(tmodel)
  generator = CSVGenerator('Pistachio-Counting/pesteh_set/test{}.csv'.format(fold_num),'Pistachio-Counting/pesteh_set/classes.csv')
  ap, f1_score, accuracy,recall,precision,TPL,FPL=evaluate(generator,model)

  ap_zero=ap[0][0] # average precision for class 1
  ap_one=ap[1][0] # average precision for class 2
  f1_zero=f1_score[0]
  f1_one=f1_score[1]
  acc_zero=accuracy[0]
  acc_one=accuracy[1]
  recall_zero=recall[0]
  recall_one=recall[1]
  prec_zero=precision[0]
  prec_one=precision[1]
  tp_zero=TPL[0]
  tp_one=TPL[1]
  fp_zero=FPL[0]
  fp_one=FPL[1]
  TP=tp_zero+tp_one
  FP=fp_zero+fp_one
  number_zero=ap[0][1]
  number_one=ap[1][1]
  fn_zero=number_zero-tp_zero
  fn_one=number_one-tp_one
  FN=fn_zero+fn_one
  RECALL=TP/(TP+FN)
  PRECISION=TP/(TP+FP)
  F1_SCORE=(2*PRECISION*RECALL)/(PRECISION+RECALL)
  ACCURACY=TP/(TP+FP+FN)


  total_instances = []
  precisions = []
  for label, (average_precision, num_annotations) in ap.items():
      total_instances.append(num_annotations)
      precisions.append(average_precision)
  MAP = sum(precisions) / sum(x > 0 for x in total_instances) #calculate the mean average precision


  data[model_name]=[fold_num,ap_zero,ap_one,f1_zero,f1_one,acc_zero,acc_one,recall_zero,recall_one,prec_zero,prec_one,
                    tp_zero,tp_one,fp_zero,fp_one,fn_zero,fn_one,TP,FP,FN,MAP,F1_SCORE,ACCURACY,RECALL,PRECISION,number_zero,number_one,number_zero+number_one]
with open('Pistachio_detection.csv', mode='w',newline='') as csv_file: # write the results to a csv file
    csvwriter = csv.writer(csv_file, delimiter=',', quotechar='"',quoting=csv.QUOTE_MINIMAL)
    for key in data:
        write_data=[key]
        write_data.extend(data[key])
        csvwriter.writerow(write_data)